# Downloading GUNW products using productAPI.py

**Author**: Brett A. Buzzanga, David Bekaert - Jet Propulsion Laboratory

This notebook demonstrates how to use the productAPI.py command line tool to download Sentinel 1 ARIA Geocoded UNWrapped interferogram (**GUNW**) products.  A detailed overview of the ARIA GUNW product with respect to processing, formatting, sampling, and data layers can be found on the [ARIA website](https://aria.jpl.nasa.gov/node/97).

The **`ariaDownload.py`** program wraps around the NASA's ASF DAAC API and Bulk Download Service (https://bulk-download.asf.alaska.edu/help). The ASF Bulk Download Service handles most of the heavy lifting of the data-download and will conveniently skip previously downloaded files, and re-download partially downloaded files.  
In this notebook, we will demonstrate **`ariaDownload.py`** functionality along track 4, which intersects the U.S. East Coast in southeastern Virginia.

<div class="alert alert-warning">
<b>Potential download failure:</b>
GUNW products are hosted at the NASA ASF DAAC. Downloading them requires a NASA Earthdata URS user login and requires users to add “ARIA Product Search” to their URS approved applications
</div>

<div class="alert alert-danger">
<b>Download</b>:     
    
- The ***jupuyter notebook* does not allow for interactive entering of your user-name and password, use the *jupyter terminal* instead** with the same command for interactive use.
</div>

## Overview of the ariaDownload.py program

Running **`ariaDownload.py`** with no options, or with **`-h`**, will show the parameters options as well as some examples. At minimum, users need to specify a spatial constraint either as a track number or bounding box (can be a shapefile).

Let us explore what some of the other options are:

In [1]:
!ariaDownload.py

usage: ariaDownload.py [-h] [-o OUTPUT] [-t TRACK] [-b BBOX] [-w WD]
                       [-s START] [-e END] [-l DAYSLT] [-m DAYSGT] [-i IFG]
                       [-d FLIGHTDIR] [-v]

Program to download all GUNW products for a single track.

optional arguments:
  -h, --help            show this help message and exit
  -o OUTPUT, --output OUTPUT
                        Output type, default is "Download". "Download",
                        "Count", and "Kml" are currently supported.
  -t TRACK, --track TRACK
                        track to download
  -b BBOX, --bbox BBOX  Lat/Lon Bounding SNWE
  -w WD, --workdir WD   Specify directory to deposit all outputs. Default is
                        "products" in local directory where script is
                        launched.
  -s START, --start START
                        Start date as YYYYMMDD; If none provided, starts at
                        beginning of Sentinel record (2015).
  -e END, --end END     End date as YYYYMMDD. If 

## 1. Example of ariaDownload output options

The user has the option to **download**, **count**, and generate a **kmz** corresponding to return of their search criteria. Below we provide example for each of these over Track 4.

### Count the number of products

To get a count of the number of products, without downloading data, provide the **`--output`** option with the **`count`** argument. To get information on the exact product filenames also include the verbose option **`-v`**.

In [2]:
!ariaDownload.py --track 4 --output count

https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&output=JSON&relativeOrbit=004

Found -- 1508 -- products


### Generate KMZ of products

To generate a Google Earth kmz, without downloading data, provide the **`--output`** option with the **`kmz`** argument. In case you also want to get information on the exact product filenames you can enbale the verbose option by adding **`-v`**. The result is shown in **Fig. 1**.

In [26]:
!ariaDownload.py --track 4 --output kmz

https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&output=JSON&relativeOrbit=4
Wrote .KMZ to:
	 /Users/dbekaert/Software/ARIA_tools/ARIA-tools-docs-current/JupyterDocs/ariaDownload/products/download_products_4.kmz


<img src="./support_docs/track_004.png" alt="track" width="700">
<center><b>Fig. 1</b> Image of products available along Track 4. </center>

### Download products

To download the products, simply omit the **`--output`** option or specify it with the **`download`** argument. Examples for downloading all data on track 4 include:

`ariaDownload.py -t 004`

`ariaDownload.py -t 004 -o download`


By default, the products will be downloaded into the *./products* folder. You can specify a different location using the **`-w`** option, such as:

`ariaDownload.py -t 004 -w /insarHome`

<div class="alert alert-warning">
<b>Potential download failure:</b>
GUNW products are hosted at the NASA ASF DAAC. Downloading them requires a NASA Earthdata URS user login and requires users to add “ARIA Product Search” to their URS approved applications
</div>

## 2. Spatial-temporal subsetting

The **`ariaDownload.py`** program has a number of options availble for subsetting the search of products in the spatial and temporal domain, including **bounding box**, **start/end date**, **aquisition geometry**, **temporal baseline**, or **interferogram** options. Below, using the count option for brevity, we demonstrate each of them.

### Subsetting by Bounding Box

Bounding box can either be specified as **SNWE** coordinates (a string with quotation) or by providing a **shapefile** (GeoJSON or ESRI shapefiles) to the **`--bbox`** option. 

An example based on subsetting using **SNWE** coordinates and the corresponding kmz shown in **Fig. 2**:

In [7]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count

https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&output=JSON&relativeOrbit=4&bbox=-76.655,36.75,-75.928,37.225

Found -- 214 -- products


<img src="./support_docs/crop_hr.png" alt="crop" width="700">

<center><b>Fig. 2</b> Image of products available along within bounding box 36.75S, 37.225N, -76.655W, -75.928E</center>

An example based on subsetting using s shape file:

In [9]:
!ariaDownload.py --track 4 --bbox './support_docs/HR_North.shp' -o count

https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&output=JSON&relativeOrbit=4&bbox=-76.6790935044502,36.9347870763244,-76.0116546923148,37.4784939161566

Found -- 427 -- products


<img src="./support_docs/crop_shp.png" alt="crop" width="700">

<center><b>Fig. 3</b> Image of products available along within shapefile (blue shading). </center>

### Subsetting by start/end date

In time, subsetting can be done by specifying a range in which products need to fall. This is controlled using the **`--start YYYYMMDD` ** and **`--end YYYYMMDD`** options. By default the complete observational record is considered.

In [14]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --start 20190101 

https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&output=JSON&relativeOrbit=4&bbox=-76.655,36.75,-75.928,37.225

Found -- 18 -- products


In [15]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count -s 20190101  --end 20190401 --verbose

https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&output=JSON&relativeOrbit=4&bbox=-76.655,36.75,-75.928,37.225
Found: S1-GUNW-A-R-004-tops-20190325_20190313-230628-37654N_35777N-PP-7e46-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190325_20190301-230628-37654N_35777N-PP-13e7-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190313_20190217-230627-37655N_35778N-PP-7b05-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190313_20190301-230627-37655N_35778N-PP-e293-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190301_20190124-230628-37655N_35778N-PP-d26d-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190217_20190124-230628-37654N_35778N-PP-7fcc-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190217_20190112-230628-37654N_35778N-PP-0810-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190124_20190112-230628-37654N_35778N-PP-2cb6-v2_0_2

Found -- 8 -- products


Note the use of the **`--verbose`** option used to show the products found.

### Subsetting based on product temporal baseline

The **`--daysless`** and **`--daysmore`** options allow to subset based on the temporal baseline, i.e. the time length (in days) between the reference and secondary SAR acquisitions of the interferogram. Note that `--daysless 12` and `--daysmore 12` means respectively baselines smaller and larger than 12 (not including 12 itself).

To select only interferograms products with temporal baselines shorter than 13 days:

In [16]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --daysless 13 -v

https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&output=JSON&relativeOrbit=4&bbox=-76.655,36.75,-75.928,37.225
Found: S1-GUNW-A-R-004-tops-20190512_20190430-230629-37655N_35779N-PP-3cc1-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190418_20190406-230628-37654N_35777N-PP-b59c-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190406_20190325-230628-37654N_35777N-PP-26cf-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190325_20190313-230628-37654N_35777N-PP-7e46-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190313_20190301-230627-37655N_35778N-PP-e293-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190124_20190112-230628-37654N_35778N-PP-2cb6-v2_0_2
Found: S1-GUNW-A-R-004-tops-20190112_20181231-230628-37655N_35778N-PP-b6d0-v2_0_2
Found: S1-GUNW-A-R-004-tops-20181231_20181219-230629-37655N_35778N-PP-5307-v2_0_2
Found: S1-GUNW-A-R-004-tops-20181125_20181113-230630-37655N_35778N-PP-f6d6-v2_0_2
Found: S1-GUNW-A-R-004-tops-20180902_20180821-230630-37655N_35778N-PP-6efa-v2_0_2
Found: S1-GUNW-A

Select only pairs with temporal baselines, greater than a year:

In [18]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --daysmore 364

https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&output=JSON&relativeOrbit=4&bbox=-76.655,36.75,-75.928,37.225

Found -- 113 -- products


Select only pairs with moderate temporal baselines between two and six months:

In [19]:
!ariaDownload.py --track 4  --bbox "36.75 37.225 -76.655 -75.928"  -o count -m 59 -l 181 -v 

https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&output=JSON&relativeOrbit=4&bbox=-76.655,36.75,-75.928,37.225
Found: S1-GUNW-A-R-004-tops-20161018_20160702-230618-37656N_35619N-PP-199b-v2_0_2
Found: S1-GUNW-A-R-004-tops-20161006_20160702-230618-37655N_35618N-PP-6382-v2_0_2
Found: S1-GUNW-A-R-004-tops-20161006_20160409-230618-37655N_35618N-PP-8288-v2_0_2
Found: S1-GUNW-A-R-004-tops-20160702_20160409-230614-37655N_35618N-PP-f253-v2_0_2
Found: S1-GUNW-A-R-004-tops-20150825_20150310-230613-37822N_35782N-PP-40a2-v2_0_1

Found -- 5 -- products


### Subsetting based on interferogram pair

To find a specific interferogram combination one can use the **`--ifg YYYYMMDD_YYYYMMDD`** option.

In [25]:
!ariaDownload.py -b "36.75 37.225 -76.655 -75.928"  -o count --ifg "20161018_20160702" -v 

https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&output=JSON&relativeOrbit=4&bbox=-76.655,36.75,-75.928,37.225
Found: S1-GUNW-A-R-004-tops-20161018_20160702-230618-37656N_35619N-PP-199b-v2_0_2

Found -- 1 -- products
